In [1309]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)
library(ggplot2)
library(dummies)
library(stringr)
library(rms)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_score_finale.Rdata")
df <- df_score_finale
df <- subset(df,select= -c(degenzaPostTraOSP))

In [1310]:
names(df) <- c('Centre', 'Gender', 'Age', 'LHOSpretx', 'Status', 'VaricealBleed', 'Ascites', 'Tips',  
                'PVthrombosis', 'NutritStatus', 'Previoussurg', 'Ventilationpretx', 'Vasopressors', 
               'InfusionAlb', 'PaO2FiO2', 'PlatCount', 'Albumin', 'Duration', 'PRBCintraop', 'FFPintraop', 'Plateletsintraop',
               'Fibrinogen', 'CRIO', 'Crystalloids', 'PPS5', 'Norepinend', 'Lactates', 'ColdIschemia', 'ReperfusSy', 'PCshunt',
               'Cavaveinanasth', 'VVBypass', 'Biliaryanasth', 'Openabdomen', 'Donorgender', 'Donortype', 'Organtype' ,'DonorECMO', 
               'RegionalPerf', 'OrganBiopsy', 'Machineperf', 'DonorSite', 'DonorCauseDeath', 'DonorAge', 'DonorHeight', 'DonorSodiemia',
               'DonorGGT', 'MELDNa', 'Alcoholic', 'cirrhosisDrugs', 'NASH', 'Lateretx', 'DonorMaastricht', 'DonorSteatosis' ,
               'HypoNormoter', 'Nocomorbodities', 'Hypertension', 'SeverePsychosis', 'Periodstx', 'Nighttx', 'Admsite' ,
               'Encephalopathy', 'Cholestatic', 'Rare', 'Viral', 'HCC', 'Respcomorbidities', 'CVcomorbidities', 'Neurocomorbidities',
               'Renalcomorbidities', 'Diabetes', 'Anticoagulation', 'MELDExcept', 'deg_cat')

In [1001]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +42 secs: 

 confirmed 11 attributes: Albumin, Centre, Duration, Lactates, LHOSpretx and 6 more;

 rejected 39 attributes: Age, Alcoholic, Anticoagulation, Cholestatic, cirrhosisDrugs and 34 more;

 still have 23 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +50 secs: 

 confirmed 2 attributes: Ascites, PPS5;

 rejected 5 attributes: DonorGGT, Donortype, Organtype, Viral, VVBypas

 [1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Admsite"          "Encephalopathy"   "HCC"             


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [600]:
par(mar = c(13, 5, 2, 0.1))  
plot(boruta_output, cex.axis=1.3, las=2, xlab="", main="Variable Importance")

In [818]:
df_boruta$deg_cat <- df$deg_cat

In [1090]:
boruta_signif #variabili selezionate

[1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "CRIO"             "Crystalloids"     "PPS5"             "Norepinend"      
[17] "Lactates"         "ReperfusSy"       "Openabdomen"      "DonorSodiemia"   
[21] "MELDNa"           "Admsite"          "Encephalopathy"   "HCC"

In [1311]:
df$deg_cat_bin = ifelse(df$deg_cat == 'fast',1,0)

df <- subset(df, select = -deg_cat)

In [1312]:
df <- df[df$PRBCintraop < 52,] # 4 big outliers

In [1313]:
df <- subset(df, select = c('MELDNa','PRBCintraop','deg_cat_bin'))

In [1314]:
df$dataset <- sample(x=c("train","validate"),
size=nrow(df),
replace=TRUE,
prob=c(0.75,0.25))

## LOESS smoothing curves for numeric variables

In [1315]:
df.cont<- df[df$dataset=="train",
c('deg_cat_bin','MELDNa','PRBCintraop')]




ymark<-seq(0,1,0.1)

for(var in names(df.cont)[-1]){
xvar<-seq(min(df.cont[,var]),
max(df.cont[,var]),
length.out=10)
    
    
mypath <- file.path("C:/Users/abonini/Documents/score",
paste(paste("plot",var,sep = "_"),
"pdf", sep = "."))

pdf(file=mypath)

gg<-ggplot(df.cont, aes(x=df.cont[,var],y=deg_cat_bin))+
geom_jitter(size=1, alpha=0.2,height=0.05)+
stat_smooth(method="loess",colour="blue", size=1.5)+
xlab(var)+
ylab("Probability on length of hospital stay")+
theme_bw()+
geom_hline(yintercept=ymark,col="red")+
scale_y_continuous(breaks=ymark)+
geom_vline(xintercept=xvar,col="green",alpha=0.5)+
scale_x_continuous(breaks=round(xvar,2))
    
print(gg)
dev.off()
}

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'



## Choosing cut points and base reference

In [1316]:
MELDNacut<-c(12,15,30)
MELDNab<-"[2.05,12]"

PRBCintraopcut <-c(3,5,7)
PRBCintraopb<-"[0,3]"




cont.to.cat<-data.frame(id=c(1:nrow(df.cont)))

for (var in names(df.cont)[-1]) {
cat<-cut(df.cont[,var],
breaks=c(min(df.cont[,var]),
get(paste(var,"cut",sep="")),
max(df.cont[,var])),include.lowest= TRUE)
cat<-relevel(cat,ref=get(paste(var,"b",sep="")))
cont.to.cat<-cbind(cont.to.cat,cat)
}

## Calculating scores for each level

In [1317]:
df.cont.to.cat<-cont.to.cat[,-1] 

names(df.cont.to.cat)<-names(df.cont)[-1]

df.final<-cbind(df.cont.to.cat,

deg_cat_bin=df.cont$deg_cat_bin)

mod<-glm(deg_cat_bin~.,
df.final, family="binomial")
score<-coef(mod)[-1]


score.cont<-score

In [1318]:
summary(mod)


Call:
glm(formula = deg_cat_bin ~ ., family = "binomial", data = df.final)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3218  -0.8306  -0.5249   1.0395   2.1189  

Coefficients:
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)         0.3333     0.1180   2.824  0.00474 ** 
MELDNa(12,15]      -0.4214     0.2174  -1.938  0.05261 .  
MELDNa(15,30]      -1.2204     0.1736  -7.029 2.08e-12 ***
MELDNa(30,48.3]    -2.1769     0.4258  -5.112 3.18e-07 ***
PRBCintraop(3,5]   -0.2892     0.2029  -1.425  0.15404    
PRBCintraop(5,7]   -0.9255     0.2949  -3.138  0.00170 ** 
PRBCintraop(7,44]  -1.0254     0.2632  -3.896 9.80e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1220.2  on 944  degrees of freedom
Residual deviance: 1070.7  on 938  degrees of freedom
AIC: 1084.7

Number of Fisher Scoring iterations: 5


In [1319]:
head(df.final)

MELDNa,PRBCintraop,deg_cat_bin
"[2.05,12]","(3,5]",1
"(15,30]","(7,44]",0
"[2.05,12]","[0,3]",1
"[2.05,12]","[0,3]",1
"(30,48.3]","(5,7]",0
"[2.05,12]","[0,3]",0


## Calculating scores for numeric variables

In [1320]:
var.cont<-as.character(1)
#2
for(var in names(score.cont)){
#3
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
#4
var.cont<-c(var.cont,var.red)
#5
}
#6
var.cont<-unique(var.cont)[-1]
#7
for(var in var.cont){
#8
df[,paste(var,"points",sep=".")]<-as.numeric(NA)
#9
}
#10
for (var in names(score.cont)){
#11
var.red<-sub("(\\(|\\[)[0-9]+.*", "", var)
#12
var.low<-as.numeric(str_extract(var,'(?<=(\\(|\\[))[0-9]+\\.*[0-9]*(?=\\,)'))
#13
var.upper<-as.numeric(str_extract(var,'(?<=\\,)[0-9]+\\.*[0-9]*(?=\\])'))
#14
df[,paste(var.red,"points",sep=".")]<-ifelse(
#15
df[,var.red]<=var.upper&df[,var.red]>=var.low,
#16
score[var],df[,paste(var.red,"points",sep=".")])
#17
}
#18
for(var in var.cont){
#19
df[,paste(var,"points",sep=".")]<-ifelse(
#20
is.na(df[,paste(var,"points",sep=".")]),
#21
0,df[,paste(var,"points",sep=".")]
#22
)
#23
}
#24

# Calculate scores for factor variables and take a sum

var.cat<-names(df.cat)

for(var in var.cat){
    df[,paste(var,"points",sep=".")]<-as.numeric(NA)
}

for (var in var.cat){
score.var<-score.cat[grep(var,names(score.cat))]
        names(score.var)<-sub(var,"",names(score.var))
    for(i in 1:(length(levels(df[,var]))-1)){
        df[,paste(var,"points",
        sep=".")]<-ifelse(
        df[,var]==names(score.var)[i]&
        is.na(df[,paste(var,"points",sep=".")]),
        score.var[i],
    df[,paste(var,"points",sep=".")])}
    }

for(var in var.cat){
    df[,paste(var,"points",sep=".")]<-ifelse(
    is.na(df[,paste(var,"points",sep=".")]),
    0,df[,paste(var,"points",sep=".")])}

### Non uso categoriche

In [1321]:
df$score<-rowSums(df[,grepl("\\.+points",names(df))])

In [1322]:
head(df)

,MELDNa,PRBCintraop,deg_cat_bin,dataset,MELDNa.points,PRBCintraop.points,score
NA.575,10.26,5,1,train,0.000000,-0.9254543,-0.9254543
NA.577,26.68,44,0,train,-1.220383,-1.0253695,-2.2457525
NA.682,4.44,0,1,train,0.000000,0.0000000,0.0000000
NA.687,25.78,14,0,validate,-1.220383,-1.0253695,-2.2457525
NA.692,4.14,0,1,train,0.000000,0.0000000,0.0000000
NA.693,7.29,1,1,validate,0.000000,0.0000000,0.0000000


## Converting the score to probability of mortality and comparing it to observed number of deaths

In [1323]:
glmod<-glm(deg_cat_bin~score,
df[df$dataset=="train",],
family="binomial")

newx<-seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score))

prd<-predict(glmod,
newdata=data.frame(score=newx),
type="response",
se.fit=T)

count<-as.matrix(table(cut(df[df$dataset=="train",]$score,
breaks=seq(min(df[df$dataset=="train",]$score),
max(df[df$dataset=="train",]$score)),
include.lowest = T),
df[df$dataset=="train",]$deg_cat_bin))


In [1324]:
summary(glmod)


Call:
glm(formula = deg_cat_bin ~ score, family = "binomial", data = df[df$dataset == 
    "train", ])

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3403  -0.8605  -0.5551   1.0228   2.3193  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  0.37522    0.10850   3.458 0.000543 ***
score        0.96523    0.08741  11.042  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1220.2  on 944  degrees of freedom
Residual deviance: 1071.0  on 943  degrees of freedom
AIC: 1075

Number of Fisher Scoring iterations: 4


## Visualization of the relationship between scores and probability of outcome events

In [1325]:
par(mar=c(5,4,4,5)+.1)

barplot(t(count),
main="Scores versus probability on length of hospital stay",
xlab="Scores",
ylab="Observed number of patients",
space=0,
col=c("yellow","lightblue"))

legend("topleft",fill=c("yellow","lightblue",NA),
lty = c(NA,NA,1),lwd=c(NA,NA,2),
legend=c("fast","NotFast",
"Predicted Prob."),
col=c("black"),
border = c("black","black",NA))

par(new=TRUE)

plot(prd$fit~newx,
type="l",col="black",
lwd=2,xaxt="n",yaxt="n",
xlab="",ylab="")

polygon(c(rev(newx), newx),
c(rev(prd$fit+1.96*prd$se.fit),
prd$fit-1.96*prd$se.fit),
col = adjustcolor('grey80',alpha=0.5),
border = NA)

lines(newx, prd$fit+1.96*prd$se.fit,
lty = 'dashed', col = 'red')

lines(newx, prd$fit-1.96*prd$se.fit,
lty = 'dashed', col = 'red')

axis(4)

mtext("Predicted length of hospital stay",
side=4,line=3)


## Validation of the score

In [1326]:
ddist <- datadist(df)

options(datadist='ddist')

f.score<-lrm(deg_cat_bin~score,
df[df$dataset=="train",],
x=TRUE,y=TRUE)

phat.score<-predict(f.score,
df[df$dataset=="validate",],
type="fitted")

v.score<-val.prob(phat.score,
df[df$dataset=="validate",]$deg_cat_bin,
m=20)


Warning message in min(xx[xx > upper]):
"no non-missing arguments to min; returning Inf"


# Prediction

In [1327]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

In [1328]:
df$deg_cat_bin <- ifelse(df$deg_cat_bin == 1,"fast","notFast")

In [1329]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [1330]:
df_train <- df[df$dataset == "train",]

In [1331]:
df_train <- SmoteClassif(deg_cat_bin ~ score, df_train, k = 27, dist = "Manhattan")

#view distribution of response variable in new dataset
table(df_train$deg_cat_bin)


Warning message in Smote.exsClassif(dat[which(dat[, ncol(dat)] == li[[1]][ove[i]]), :
"si è prodotto un NA per coercizione"
Warning message in lapply(X = X, FUN = FUN, ...):
"si è prodotto un NA per coercizione"


ERROR: Error in neighbours(tgt, dat, dist, p, k): Can not compute Manhattan distance with nominal attributes!


In [1332]:
df$score

[1] -0.9254543 -2.2457525  0.0000000 -2.2457525  0.0000000  0.0000000
   [7] -3.1023900 -3.1023900  0.0000000 -2.1458372 -1.5095496  0.0000000
  [13]  0.0000000 -0.4213746 -2.1769356 -2.2457525 -0.4213746 -1.2203829
  [19]  0.0000000 -0.4213746 -2.1458372 -0.2891666  0.0000000 -1.4467441
  [25] -0.4213746 -2.2457525 -2.1458372  0.0000000 -2.2457525 -1.3468289
  [31]  0.0000000  0.0000000 -0.4213746  0.0000000 -0.2891666 -0.9254543
  [37]  0.0000000 -0.4213746 -2.1458372 -2.1458372 -2.2457525 -2.2457525
  [43] -2.2457525  0.0000000 -1.2203829 -0.2891666 -2.2457525 -1.3468289
  [49] -1.5095496 -1.2203829 -0.9254543 -1.3468289 -3.2023052  0.0000000
  [55]  0.0000000 -0.9254543 -0.2891666 -2.2457525  0.0000000  0.0000000
  [61] -1.0253695  0.0000000 -3.1023900  0.0000000 -3.1023900  0.0000000
  [67] -2.4661023 -3.2023052 -0.2891666 -0.9254543 -1.2203829 -0.7105412
  [73] -1.2203829 -1.3468289 -0.9254543  0.0000000 -3.2023052  0.0000000
  [79] -2.1458372 -0.4213746 -1.5095496  0.0000000 -3.2023052 -1.2203829
  [85] -2.1458372 -2.2457525  0.0000000 -2.1458372 -1.2203829  0.0000000
  [91] -0.2891666  0.0000000 -1.5095496 -1.4467441  0.0000000 -0.7105412
  [97] -0.2891666  0.0000000  0.0000000 -0.4213746  0.0000000 -1.0253695
 [103]  0.0000000 -1.5095496  0.0000000  0.0000000 -1.2203829 -0.7105412
 [109] -0.7105412  0.0000000 -2.1458372 -3.1023900 -0.4213746 -2.4661023
 [115]  0.0000000  0.0000000  0.0000000 -0.4213746 -0.4213746  0.0000000
 [121] -1.2203829  0.0000000 -1.2203829 -0.4213746 -2.1769356 -0.4213746
 [127]  0.0000000 -0.4213746  0.0000000 -1.2203829 -3.1023900 -2.1458372
 [133] -1.2203829 -1.5095496  0.0000000  0.0000000  0.0000000  0.0000000
 [139] -0.4213746 -2.1458372  0.0000000 -1.2203829 -0.4213746 -3.2023052
 [145] -3.2023052 -2.2457525 -3.1023900 -1.5095496  0.0000000 -0.4213746
 [151] -1.5095496  0.0000000 -0.4213746 -1.2203829 -1.2203829 -1.2203829
 [157] -2.4661023 -1.5095496 -1.2203829 -1.2203829 -2.2457525 -1.2203829
 [163] -2.1458372 -2.2457525 -2.2457525 -0.2891666  0.0000000  0.0000000
 [169] -2.1458372 -1.5095496 -3.1023900 -1.5095496 -1.2203829 -1.2203829
 [175] -2.4661023 -1.2203829 -1.5095496 -3.2023052 -0.2891666 -1.2203829
 [181]  0.0000000 -1.5095496 -0.2891666 -1.0253695 -0.4213746 -1.5095496
 [187] -3.1023900  0.0000000 -1.2203829  0.0000000 -0.2891666  0.0000000
 [193]  0.0000000 -2.4661023 -1.5095496 -2.1458372 -2.4661023 -3.2023052
 [199] -1.2203829  0.0000000 -1.5095496 -2.1458372 -2.2457525 -0.4213746
 [205] -2.1458372  0.0000000 -2.2457525 -1.0253695 -3.2023052 -2.2457525
 [211] -0.9254543 -1.2203829 -0.2891666  0.0000000 -1.4467441 -1.4467441
 [217]  0.0000000  0.0000000 -2.1458372 -2.2457525 -1.4467441  0.0000000
 [223] -0.4213746 -2.2457525 -3.2023052  0.0000000 -0.2891666 -0.9254543
 [229] -2.1458372 -3.2023052 -2.1458372 -1.5095496 -2.2457525  0.0000000
 [235]  0.0000000 -1.0253695 -1.5095496 -2.2457525  0.0000000 -1.5095496
 [241] -1.0253695  0.0000000 -3.2023052  0.0000000 -0.2891666 -2.2457525
 [247]  0.0000000 -0.4213746 -1.2203829 -0.4213746 -0.9254543 -2.2457525
 [253]  0.0000000  0.0000000 -2.2457525 -0.4213746 -3.2023052  0.0000000
 [259] -2.2457525 -0.2891666 -0.4213746 -0.2891666 -1.2203829 -1.4467441
 [265]  0.0000000 -2.2457525 -2.2457525  0.0000000 -1.2203829  0.0000000
 [271]  0.0000000  0.0000000  0.0000000 -0.9254543 -1.2203829 -1.5095496
 [277] -3.2023052 -3.2023052 -0.9254543 -1.3468289 -3.2023052 -0.9254543
 [283]  0.0000000  0.0000000  0.0000000 -2.2457525  0.0000000 -1.0253695
 [289] -1.0253695 -2.2457525 -2.2457525 -1.5095496 -3.2023052 -1.2203829
 [295] -1.5095496 -0.4213746 -1.3468289 -2.2457525 -2.1458372 -2.1458372
 [301] -2.2457525 -2.4661023 -2.2457525 -2.2457525 -2.2457525 -1.0253695
 [307] -0.2891666  0.0000000 -1.5095496 -3.2023052  0.0000000 -2.2457525
 [313] -1.2203829 -2.1458372 -2.4661023  0.0000000 -3.2023052 -2.2457525
 [319] -2.1458372 -2.1458372 -2.2457525  0.0000000 -0.4213746 -2.2457525
 [325] -2.1458372 -2.2457525 -1.0253695 -2.2457525  0.0000000

In [1333]:
table(df_train$deg_cat_bin)


   fast notFast 
    328     617 

In [1334]:
#Regularized Logistic Regression
model_regLogistic <- train(deg_cat_bin ~ score , data = df_train,
               method = "regLogistic",
               trControl = ctrl)    # metric = "..." ciò che voglio massimizzare

+ Fold01.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
- Fold01.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
+ Fold01.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
- Fold01.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
+ Fold01.Rep1: cost=0.018786, loss=L1, epsilon=1 
- Fold01.Rep1: cost=0.018786, loss=L1, epsilon=1 
+ Fold02.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
- Fold02.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
+ Fold02.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
- Fold02.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
+ Fold02.Rep1: cost=0.018786, loss=L1, epsilon=1 
- Fold02.Rep1: cost=0.018786, loss=L1, epsilon=1 
+ Fold03.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
- Fold03.Rep1: cost=0.001162, loss=L2_primal, epsilon=1 
+ Fold03.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
- Fold03.Rep1: cost=0.002923, loss=L2_primal, epsilon=1 
+ Fold03.Rep1: cost=0.018786, loss=L1, epsilon=1 
- Fold03.Rep1: cost=0.018786, loss=L1, epsilon=1 
+ Fold04.Rep1: c

In [1335]:
print(model_regLogistic)
plot(model_regLogistic)

Regularized Logistic Regression 

945 samples
  1 predictor
  2 classes: 'fast', 'notFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 851, 851, 850, 851, 851, 851, ... 
Resampling results across tuning parameters:

  cost         loss       Accuracy   Kappa     
  0.001162181  L2_primal  0.6529115  0.00000000
  0.002923303  L2_primal  0.6529115  0.00000000
  0.018785553  L1         0.6561030  0.02582851

Tuning parameter 'epsilon' was held constant at a value of 1
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were cost = 0.01878555, loss = L1
 and epsilon = 1.


In [1336]:
df_test <- df[df$dataset == "validate",]
test <- df_test[,"deg_cat_bin"]
test <- as.factor(test)

In [1337]:
pred_regLogistic <- predict(model_regLogistic,df_test)  
pred_regLogistic.prob <- predict(model_regLogistic,df_test, type="prob")


result <- confusionMatrix(test,  pred_regLogistic) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast notFast
   fast       0      93
   notFast    0     219
                                          
               Accuracy : 0.7019          
                 95% CI : (0.6478, 0.7521)
    No Information Rate : 1               
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity :     NA          
            Specificity : 0.7019          
         Pos Pred Value :     NA          
         Neg Pred Value :     NA          
             Prevalence : 0.0000          
         Detection Rate : 0.0000          
   Detection Prevalence : 0.2981          
      Balanced Accuracy :     NA          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
                  NA            0.7019231                   NA 
      Neg Pred Value            Precision               Recall 
                  NA            0.0000000                   NA 
                  F1           Prevalence       Detection Rate 
                  NA            0.0000000            0.0000000 
Detection Prevalence    Balanced Accuracy 
           0.2980769                   NA